In [1]:
import numpy as np
import cv2
#import pandas as pd
import os
import tifffile as tiff
from PIL import Image
import skimage
import czifile
import matplotlib.pyplot as plt
from aicsimageio import AICSImage
import json

## CZI to TIFF

In [3]:
def czi_to_tiff(czi_folder,save_path):
    """read czi files and save as tiff files. CZYX -> YXC"""
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for root, dirs, files in os.walk(czi_folder):
        for file in files:
            if file.endswith('.czi'):
                czi = AICSImage(os.path.join(root,file))
                image = czi.get_image_data("CZYX", S=0, T=0, Z=0)
                image = np.squeeze(image)
                image = np.moveaxis(image, 0, -1)
                image[...,2] = 0
                tiff.imwrite(os.path.join(save_path,file.replace('.czi','.tif')),image[...,:3])

czi_to_tiff(czi_folder='/Users/tom/Downloads/fiji-test',
            save_path='/Users/tom/Downloads/fiji-test_tiff')


In [5]:
def czi_to_tiff(czi_folder,save_path):
    """read czi files and save as tiff files. CZYX -> YXC"""
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for root, dirs, files in os.walk(czi_folder):
        for file in files:
            if file.endswith('.czi'):
                czi = AICSImage(os.path.join(root,file))
                image = czi.get_image_data("CZYX", S=0, T=0, Z=0)
                image = np.squeeze(image)
                image = np.moveaxis(image, 0, -1)
                image[...,2] = 0
                new_image = np.stack([image[...,0],image[...,3],np.zeros_like(image[...,0])],axis=-1)
                tiff.imwrite(os.path.join(save_path,file.replace('.czi','.tif')),new_image)

czi_to_tiff(czi_folder='/Users/tom/Downloads/fiji-test',
            save_path='/Users/tom/Downloads/fiji-test_tifff')

## OMETIFF to TIFF

In [13]:
def ometifs_to_tifs(dir_path, save_path):
    """converts ome-tifs to tifs"""
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for file in os.listdir(dir_path):
        if file.endswith('.ome.tif'):
            img = skimage.io.imread(os.path.join(dir_path, file))
            for i in range(img.shape[0]):
                skimage.io.imsave(os.path.join(save_path, file[:-8] + f'_{i}.tif'), img[i])

input_path = '/Users/tom/Downloads/ometifs'
output_path = '/Users/tom/Downloads/tifs_from_ome'

ometifs_to_tifs(input_path, output_path)

## Normalize and convert to grayscale

In [2]:
import os
import numpy as np
import skimage
folder_path = "/Users/tom/Desktop/Stanford/RA/OligodendroSight/OL_mrcnn/data/test/imgs"
save_path_norm = "/Users/tom/Desktop/Stanford/RA/OligodendroSight/OL_mrcnn/data/test/imgs_norm"
GRAYSCALE = False
if not os.path.exists(save_path_norm):
    os.makedirs(save_path_norm)
for file in os.listdir(folder_path):
    if file.endswith('.tif'):
        original_img = skimage.io.imread(os.path.join(folder_path, file))
        if GRAYSCALE:
            float_gray_img = skimage.io.imread(os.path.join(folder_path, file), as_gray=True)
            float_gray_img_norm = skimage.exposure.equalize_adapthist(float_gray_img, clip_limit=0.04)
            img_norm = np.stack(((float_gray_img_norm * 255).astype(np.uint8),)*3, axis=-1)
        else:
            norm_channels = []
            for i in range(original_img.shape[2]):
                if np.max(original_img[:,:,i]) > 0:
                    filt_channel = skimage.filters.median(original_img[:,:,i])
                    norm_channel = skimage.exposure.equalize_adapthist(filt_channel, clip_limit=0.02)
                    norm_channel = (norm_channel * 255).astype(np.uint8)
                else:
                    norm_channel = original_img[:,:,i].astype(np.uint8)
                norm_channels.append(norm_channel)
            img_norm = np.stack(norm_channels, axis=-1)
        skimage.io.imsave(os.path.join(save_path_norm, os.path.basename(file)), img_norm)
print(f'{len(os.listdir(save_path_norm))} images were normalized and saved.')

11 images were normalized and saved.


## Mask.tif to .json for training

In [ ]:
data_path = '/Users/tom/Desktop/Stanford/RA/OligodendroSight/datasets/Lauren/'
img_path = os.path.join(data_path, 'Sample_TomGUI_norm')
mask_path = os.path.join(data_path, 'masks')
json_path = os.path.join(data_path, 'jsons')

for filename in sorted(os.listdir(img_path)):
    if filename.endswith('.tif'):
        print(filename)
        img = cv2.imread(os.path.join(img_path, filename))
        mask_color = cv2.imread(os.path.join(mask_path, filename),1)
        mask_gray = 255- cv2.imread(os.path.join(mask_path, filename),0)
        contours, _ = cv2.findContours(mask_gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
        sorted_contours = sorted(contours, key=lambda x: len(x), reverse=True)
        #print(len(sorted_contours))
        for contour in sorted_contours:
            if cv2.contourArea(contour) < 1000:
                print(cv2.contourArea(contour))
                print(contour[0][0])
                sorted_contours.remove(contour)
            cv2.drawContours(mask_color, [contour], -1, (0,0,255),2)
        '''plt.figure(figsize=(15,15))
        plt.imshow(img)
        plt.title(filename)
        plt.show()
        plt.figure(figsize=(15,15))
        plt.imshow(mask_color) '''
        shapes_dict = {'shapes': [{"points":[coord[0].tolist() for coord in contour],
                           "label":"cell",
                           "cell number":f"{i+1}"} for i, contour in enumerate(sorted_contours)],
               'imagePath':os.path.join(img_path, filename),
               'imageHeight':mask_gray.shape[0],
               'imageWidth':mask_gray.shape[1]}
        json_file = os.path.join(json_path, filename[:-4]+'.json')
        with open(json_file, 'w') as f:
            json.dump(shapes_dict, f)

## Train / valid / test split

In [70]:
from sklearn.model_selection import train_test_split
import shutil

data_folder_path = '/Users/tom/Desktop/Stanford/RA/OligodendroSight/datasets/Lauren/'
img_folder_path = os.path.join(data_folder_path, 'Sample_TomGUI_norm')
mask_folder_path = os.path.join(data_folder_path, 'masks')
json_folder_path = os.path.join(data_folder_path, 'jsons')

train_path = os.path.join(data_folder_path, 'train')
valid_path = os.path.join(data_folder_path, 'valid')
test_path = os.path.join(data_folder_path, 'test')

img_train, img_else = train_test_split(os.listdir(img_folder_path), test_size=0.4, random_state=42)
img_valid, img_test = train_test_split(img_else, test_size=0.5, random_state=42)

if not os.path.exists(train_path):
    os.makedirs(train_path)
    os.makedirs(os.path.join(train_path, 'imgs'))
    os.makedirs(os.path.join(train_path, 'masks'))
    os.makedirs(os.path.join(train_path, 'jsons'))
    os.makedirs(valid_path)
    os.makedirs(os.path.join(valid_path, 'imgs'))
    os.makedirs(os.path.join(valid_path, 'masks'))
    os.makedirs(os.path.join(valid_path, 'jsons'))
    os.makedirs(test_path)
    os.makedirs(os.path.join(test_path, 'imgs'))
    os.makedirs(os.path.join(test_path, 'masks'))
    os.makedirs(os.path.join(test_path, 'jsons'))

for img_name in img_train:
    if img_name[-4:] != '.tif':
        continue
    json_name = img_name[:-4]+'.json'
    img_path = os.path.join(img_folder_path, img_name)
    mask_path = os.path.join(mask_folder_path, img_name)
    annot_path = os.path.join(json_folder_path, json_name)

    shutil.move(img_path, os.path.join(train_path, 'imgs', img_name))
    shutil.move(mask_path, os.path.join(train_path, 'masks', img_name))
    shutil.move(annot_path, os.path.join(train_path, 'jsons', json_name))   

for img_name in img_valid:
    if img_name[-4:] != '.tif':
        continue
    json_name = img_name[:-4]+'.json'
    img_path = os.path.join(img_folder_path, img_name)
    mask_path = os.path.join(mask_folder_path, img_name)
    annot_path = os.path.join(json_folder_path, json_name)

    shutil.move(img_path, os.path.join(valid_path, 'imgs', img_name))
    shutil.move(mask_path, os.path.join(valid_path, 'masks', img_name))
    shutil.move(annot_path, os.path.join(valid_path, 'jsons', json_name))

for img_name in img_test:
    if img_name[-4:] != '.tif':
        continue
    json_name = img_name[:-4]+'.json'
    img_path = os.path.join(img_folder_path, img_name)
    mask_path = os.path.join(mask_folder_path, img_name)
    annot_path = os.path.join(json_folder_path, json_name)

    shutil.move(img_path, os.path.join(test_path, 'imgs', img_name))
    shutil.move(mask_path, os.path.join(test_path, 'masks', img_name))
    shutil.move(annot_path, os.path.join(test_path, 'jsons', json_name))
